In [1]:
! pip install -U google.ai.generativelanguage

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.8/875.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 7.0 MB/s eta 0:00:00 0:00:01
Using cached pyasn1_modules-0.4.1-py3-none-any.whl (181 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached rsa-4.9-py3-none-any.whl (34 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)


In [2]:
! pip install -U google-auth-oauthlib

  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl (24 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)


In [3]:
service_account_file_name = 'service_account_key.json'

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_file_name)

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])


In [4]:
! pip install google-labs-html-chunker

  Using cached google_labs_html_chunker-0.0.5-py3-none-any.whl.metadata (4.2 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached google_labs_html_chunker-0.0.5-py3-none-any.whl (5.9 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)


In [6]:
from gemini_aqa_client import GeminiAQA

In [18]:
aqa = GeminiAQA()
lds = aqa.get_corpus("corpora/lds-3jy6n1g0gn9r")
# print(aqa.list_corpora())



In [19]:
doc = lds.create_document("BookOfMormon", metadata={"book":"Book of Mormon"})



['The Book of Mormon', 'The Book of Mormon', 'THE BOOK OF MORMON:  An Account Written by THE HAND OF MORMON UPON PLATES TAKEN FROM THE PLATES OF NEPHI Wherefore, it is an abridgment of the record of the people of Nephi, and also of the Lamanites -- Written to the Lamanites, who are a remnant of the house of Israel; and also to Jew and Gentile -- Written by way of commandment, and also by the spirit of prophecy and of revelation -- Written and sealed up, and hid up unto the Lord, that they might not be destroyed -- To come forth by the gift and power of God unto the interpretation thereof -- Sealed by the hand of Moroni, and hid up unto the Lord, to come forth in due time by way of the Gentile -- The interpretation thereof by the gift of God.', 'An abridgment taken from the Book of Ether also, which is a record of the people of Jared, who were scattered at the time the Lord confounded the language of the people, when they were building a tower to get to heaven -- Which is to show unto t

chunks {
  name: "corpora/lds-3jy6n1g0gn9r/documents/bookofmormon-nq9zzgvjqzg7/chunks/3a37j0cy9d4d"
  data {
    string_value: "The Book of Mormon"
  }
  state: STATE_PENDING_PROCESSING
}
chunks {
  name: "corpora/lds-3jy6n1g0gn9r/documents/bookofmormon-nq9zzgvjqzg7/chunks/t7nplk66ynlt"
  data {
    string_value: "The Book of Mormon"
  }
  state: STATE_PENDING_PROCESSING
}
chunks {
  name: "corpora/lds-3jy6n1g0gn9r/documents/bookofmormon-nq9zzgvjqzg7/chunks/tlus8d00olom"
  data {
    string_value: "THE BOOK OF MORMON:  An Account Written by THE HAND OF MORMON UPON PLATES TAKEN FROM THE PLATES OF NEPHI Wherefore, it is an abridgment of the record of the people of Nephi, and also of the Lamanites -- Written to the Lamanites, who are a remnant of the house of Israel; and also to Jew and Gentile -- Written by way of commandment, and also by the spirit of prophecy and of revelation -- Written and sealed up, and hid up unto the Lord, that they might not be destroyed -- To come forth by the g

In [22]:
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=2202")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=3882")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=4872")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=154527")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=332408")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=386619")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=393452")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=397940")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=406275")
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=411709")

['The Book of Mormon', 'The Book of Mormon', 'THE TESTIMONY OF THREE WITNESSES', 'Be it known unto all nations, kindreds, tongues, and people, unto whom this work shall come: That we, through the grace of God the Father, and our Lord Jesus Christ, have seen the plates which contain this record, which is a record of the people of Nephi, and also of the Lamanites, their brethren, and also of the people of Jared, who came from the tower of which hath been spoken.  And we also know that they have been translated by the gift and power of God, for his voice hath declared it unto us; wherefore we know of a surety that the work is true.  And we also testify that we have seen the engravings which are upon the plates; and they have been shown unto us by the power of God, and not of man.  And we declare with words of soberness, that an angel of God came down from heaven, and he brought and laid before our eyes, that we beheld and saw the plates, and the engravings thereon; and we know that it is 

chunks {
  name: "corpora/lds-3jy6n1g0gn9r/documents/bookofmormon-nq9zzgvjqzg7/chunks/ybt5nhl1kt3q"
  data {
    string_value: "[ 35 ]   And he also unfolded unto them all the disadvantages they labored under, by having an unrighteous king to rule over them; [ 36 ]   Yea, all his iniquities and abominations, and all the wars, and contentions, and bloodshed, and the stealing, and the plundering, and the committing of whoredoms, and all manner of iniquities which cannot be enumerated -- telling them that these things ought not to be, that they were expressly repugnant to the commandments of God. [ 37 ]   And now it came to pass, after king Mosiah had sent these things forth among the people they were convinced of the truth of his words. [ 38 ]   Therefore they relinquished their desires for a king, and became exceedingly anxious that every man should have an equal chance throughout all the land; yea, and every man expressed a willingness to answer for his own sins."
  }
  state: STATE_PE

In [46]:
q = "Are there any additional documents associated with the Book of Mormon, that provide further detail or clarification about the experience of Oliver Cowdery, David Whitmer, and Martin Harris as witnesses?"
answer = lds.generate_answer(q, "VERBOSE")
print(answer.answer.content.parts[0].text)
print("confidence: " + str(answer.answerable_probability))

The source provided, "The Book of Mormon", does not contain any additional documents associated with the Book of Mormon that provide further detail or clarification about the experience of Oliver Cowdery, David Whitmer, and Martin Harris as witnesses.
confidence: 0.0004242501745466143
